In [7]:
import os

In [8]:
%pwd

'd:\\Profiles\\sg2755\\Desktop\\New folder\\chicken_disease_classification_project\\research'

In [9]:
os.chdir("../")

In [10]:
%pwd

'd:\\Profiles\\sg2755\\Desktop\\New folder\\chicken_disease_classification_project'

In [27]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir:Path
    training_model_path: Path
    updated_base_modal_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [28]:
from src.CNN_Classifier.constant import *
from src.CNN_Classifier.utils.common import read_yaml,create_directory
import tensorflow as tf

In [29]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directory([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directory([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        
        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        
        return prepare_callback_config
    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"chicken-fecal-images")
        create_directory([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            training_model_path=Path(training.training_model_path),
            updated_base_modal_path=Path(prepare_base_model.updated_base_modal_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )    

        return training_config
    

In [30]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [31]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only = True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
        

In [32]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_modal_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.training_model_path,
            model=self.model
        )

In [33]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )

except Exception as e:
    raise e

[2024-09-05 17:52:39,027: INFO: common: yaml file: config\config.yaml]
[2024-09-05 17:52:39,031: INFO: common: yaml file: params.yaml]
[2024-09-05 17:52:39,032: INFO: common: Created directory at:artifacts]
[2024-09-05 17:52:39,034: INFO: common: Created directory at:artifacts\prepare_callbacks\checkpoint_dir]
[2024-09-05 17:52:39,035: INFO: common: Created directory at:artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-09-05 17:52:39,038: INFO: common: Created directory at:artifacts\training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.5336 - loss: 12.5583 - val_accuracy: 0.7656 - val_loss: 1.1382
Epoch 2/10
 1/19 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 1.0000 - loss: 0.0161

d:\Profiles\sg2755\AppData\Local\miniconda3\envs\myenv\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - accuracy: 1.0000 - loss: 0.0161 - val_accuracy: 1.0000 - val_loss: 0.0017
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.6167 - loss: 7.3647 - val_accuracy: 0.3906 - val_loss: 12.4519
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 77ms/step - accuracy: 0.5000 - loss: 8.6628 - val_accuracy: 0.0000e+00 - val_loss: 11.8404
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.7595 - loss: 2.2514 - val_accuracy: 0.8594 - val_loss: 1.1040
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 77ms/step - accuracy: 0.8750 - loss: 1.6085 - val_accuracy: 0.9286 - val_loss: 0.1963
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.8668 - loss: 1.2495 - val_accuracy: 0.7188 - val_loss: 2.5137
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - accuracy: 0.9375 - loss: 0.6747 - val_accuracy: 1.0000 - val_loss: 1.1495e-06
Epoch 9/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.9040 - loss: 0.6729 - val_accuracy: 0.9219 - val_l